In [1]:
import numpy as np
import polars as pl
import os
from collections import defaultdict
from tqdm import tqdm
import zipfile
import gc

In [2]:
folder1 = "/kaggle/input/ozon-embeddings/emb1/*.parquet"
folder2 = "/kaggle/input/ozon-embeddings/emb2/*.parquet"
folder3 = "/kaggle/input/ozon-embeddings/emb3/*.parquet"
folder4 = "/kaggle/input/ozon-embeddings/emb4/*.parquet"

In [3]:
embs1 = pl.scan_parquet(folder1)
embs2 = pl.scan_parquet(folder2)
embs3 = pl.scan_parquet(folder3)
embs4 = pl.scan_parquet(folder4)

embs = pl.concat([embs1, embs2, embs3, embs4])

In [4]:
chunks_dir_path = "/kaggle/input/item-feats-files-including-the-feats-themselves/ultra_mega_chunks"

filepaths = os.listdir(chunks_dir_path)
chunks_paths = sorted([os.path.join(chunks_dir_path, filepath) for filepath in filepaths if filepath.endswith(".parquet")])

In [ ]:
chunk_n = 0
for chunk_path in tqdm(chunks_paths):
    chunk = pl.read_parquet(chunk_path)
    chunk_ids = chunk['item_id'].to_list()

    raw_embeddings = (
        embs
        .sort('index')
        .filter(pl.col('index').is_in(chunk_ids))
        .collect()
    )

    grouped_embeddings = defaultdict(list)
    
    for row in raw_embeddings.iter_rows(named=True):
        index = row['index']
        embedding = row['embedding']
        grouped_embeddings[index].append(embedding)

    text_embeds_data = []
    
    for index, vectors_list in grouped_embeddings.items():
        if len(vectors_list) == 1:
            avg_embedding = vectors_list[0]
        else:
            np_vectors = [np.array(vec, dtype=np.float64) for vec in vectors_list]
            mean_vector = np.mean(np_vectors, axis=0)
            
            max_diff = 0
            for vec in np_vectors:
                diff_norm = np.linalg.norm(vec - mean_vector)
                max_diff = max(max_diff, diff_norm)
            
            if max_diff > 1e-3:
                print(f"WARNING Item {index}: max diff from mean = {max_diff:.6f}")
            
            avg_embedding = mean_vector.astype(np.float32).tolist()
        
        text_embeds_data.append({'index': index, 'embedding': avg_embedding})

    text_embeds = pl.DataFrame(text_embeds_data)

    result = (
        chunk.lazy()
        .join(
            text_embeds.lazy(), 
            left_on='item_id',
            right_on='index',
            how='inner'
        )
        .select([
            pl.col('item_id').cast(pl.Int32),
            pl.concat_list([
                pl.col('full_embed').cast(pl.List(pl.Float32)),
                pl.col('embedding').cast(pl.List(pl.Float32))
            ]).alias('concatenated_vector')
        ])
        .collect()
    )

    result.write_parquet(f"mega_chunk_{chunk_n}.parquet")
    print(f"Chunk {chunk_n} was saved")
    chunk_n += 1

    del raw_embeddings, grouped_embeddings, text_embeds_data, text_embeds, result
    gc.collect()

  1%|          | 1/100 [00:53<1:27:45, 53.19s/it]

Chunk 0 was saved


  2%|▏         | 2/100 [02:05<1:45:19, 64.48s/it]

Chunk 1 was saved


  3%|▎         | 3/100 [02:50<1:29:33, 55.39s/it]

...

Chunk 97 was saved


 99%|█████████▉| 99/100 [1:17:22<00:51, 51.29s/it]

Chunk 98 was saved


100%|██████████| 100/100 [1:18:07<00:00, 46.88s/it]

Chunk 99 was saved


In [6]:
source_dir = '/kaggle/working/'
zip_path = 'item_mega_embs.zip'

with zipfile.ZipFile(zip_path, 'a') as zipf:
    for file in os.listdir(source_dir):
        if file.endswith('.parquet'):
            file_path = os.path.join(source_dir, file)
            
            zipf.write(file_path, arcname=file)
            
            os.remove(file_path)
            print(f"Добавлен и удален: {file}")

print("Все файлы архивированы.")

Добавлен и удален: mega_chunk_63.parquet
Добавлен и удален: mega_chunk_86.parquet
Добавлен и удален: mega_chunk_83.parquet
Добавлен и удален: mega_chunk_2.parquet
Добавлен и удален: mega_chunk_85.parquet
Добавлен и удален: mega_chunk_91.parquet
Добавлен и удален: mega_chunk_99.parquet
Добавлен и удален: mega_chunk_53.parquet
Добавлен и удален: mega_chunk_62.parquet
Добавлен и удален: mega_chunk_24.parquet
Добавлен и удален: mega_chunk_97.parquet
Добавлен и удален: mega_chunk_61.parquet
Добавлен и удален: mega_chunk_20.parquet
Добавлен и удален: mega_chunk_75.parquet
Добавлен и удален: mega_chunk_40.parquet
Добавлен и удален: mega_chunk_34.parquet
Добавлен и удален: mega_chunk_27.parquet
Добавлен и удален: mega_chunk_30.parquet
Добавлен и удален: mega_chunk_32.parquet
Добавлен и удален: mega_chunk_6.parquet
Добавлен и удален: mega_chunk_59.parquet
Добавлен и удален: mega_chunk_71.parquet
Добавлен и удален: mega_chunk_26.parquet
Добавлен и удален: mega_chunk_16.parquet
Добавлен и удален: